In [1]:
import os
from zipfile import ZipFile
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Resizing, Rescaling
from shutil import copyfile
from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

2022-08-26 18:26:14.202783: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-26 18:26:16.692351: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-26 18:26:16.695302: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-26 18:26:17.105254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-26 18:26:17.105502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesl

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



2022-08-26 18:26:17.759853: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-08-26 18:26:17.759904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2022-08-26 18:26:17.759915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2022-08-26 18:26:17.760189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-26 18:26:17.760517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-26 18:26:17.760845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-

In [2]:
# extract the data from the zip file
local_zip = 'data/kagglecatsanddogs_5340.zip'

with ZipFile(local_zip, 'r') as zip:
    zip.extractall("data/")
os.remove("data/CDLA-Permissive-2.0.pdf")
os.remove("data/readme[1].txt")

In [3]:
print(len(os.listdir('data/PetImages/Cat/')))
print(len(os.listdir('data/PetImages/Dog/')))

12501
12501


In [4]:
# create folders for the resized images
create_folder_names = ["data/resized_images"]

for dir in create_folder_names:
    try:
        os.mkdir(dir)
        print("Directory " + dir + " created")
    except:
        print("Directory " + dir + " not created")

Directory data/resized_images created


In [5]:
# resize the images and save them in the resized_images folder
from keras.preprocessing.image import load_img, img_to_array, array_to_img

def resize_image(source, destination, width, height):
    all_files = []
    type = source.split("/")[-1]
    
    for file in os.listdir(source):
        file_path = source + "/" + file
    
        if os.path.getsize(file_path) and file.endswith(".jpg"):
            all_files.append(file_path)
        else:
            print(f"{file_path} is empty")
        
    for file in tqdm(all_files):       
        img = load_img(file, target_size=(width, height))
        img = img_to_array(img)
        img = array_to_img(img)
        img.save(destination + "/" + type.lower() + "_" + file.split("/")[-1])
        
    print(f"{len(all_files)} images resized")
        

resize_image("data/PetImages/Cat", "data/resized_images/", 150, 150)
resize_image("data/PetImages/Dog", "data/resized_images/", 150, 150)

data/PetImages/Cat/Thumbs.db is empty
data/PetImages/Cat/666.jpg is empty


100%|██████████| 12499/12499 [00:28<00:00, 437.41it/s]


12499 images resized
data/PetImages/Dog/Thumbs.db is empty
data/PetImages/Dog/11702.jpg is empty


 59%|█████▉    | 7426/12499 [00:16<00:11, 442.86it/s]/home/yasin.142.tarakci/miniconda3/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 12499/12499 [00:27<00:00, 452.96it/s]

12499 images resized


In [6]:
# create folders for the train, validation and test sets
create_folder_names = [
    "data/pets_images",
    "data/pets_images/train",
    "data/pets_images/test",
    "data/pets_images/validation",
]

for dir in create_folder_names:
    try:
        os.mkdir(dir)
        print("Directory " + dir + " created")
    except:
        print("Directory " + dir + " not created")

Directory data/pets_images created
Directory data/pets_images/train created
Directory data/pets_images/test created
Directory data/pets_images/validation created


In [7]:
# split the data into train, validation and test sets
def split_data(source, train_dir, test_dir, val_dir, train_size, test_size):
    all_filess = []
    
    for file in os.listdir(source):
        file_path = source + "/" + file
        
        if os.path.getsize(file_path):
            all_filess.append(file_path)
        else:
            print(f"{file_path} is empty")
    
    n_files = len(all_filess)
    split_point = int(n_files * train_size)
    
    shuffled = random.sample(all_filess, n_files)
    train_set = shuffled[:split_point]
    others = shuffled[split_point:]
    
    n_files = len(others)
    split_point = int(n_files * (test_size / (1 - train_size)))
    
    test_set = others[:split_point]
    val_set = others[split_point:]
    
    for file in tqdm(train_set):
        copyfile(file, train_dir + "/" + file.split("/")[-1])
    
    for file in tqdm(test_set):
        copyfile(file, test_dir + "/" + file.split("/")[-1])
        
    for file in tqdm(val_set):
        copyfile(file, val_dir + "/" + file.split("/")[-1])
        
src_dir = "data/resized_images/"
train_dir = "data/pets_images/train/"
test_dir = "data/pets_images/test/"
val_dir = "data/pets_images/validation/"

train_size = 0.7
test_size = 0.15

split_data(src_dir, train_dir, test_dir, val_dir, train_size, test_size)

100%|██████████| 3751/3751 [00:00<00:00, 9816.73it/s]
